## Using Python to build a Multi-Source ETL Pipeline for Sales Data Analysis
This project demonstrates the implementation of a comprehensive ETL (Extract, Transform, Load) pipeline integrating multiple data sources into a dimensional data warehouse.
In this project, I build a data warehouse that combines sales data from MongoDB, product information from FakeStore API, and stores the transformed data in a MySQL database. The integration provides a unified view of sales transactions, product inventory, and customer information for comprehensive business analysis.

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [3]:
import os
import logging
from typing import Dict
import pandas as pd
import numpy as np
import pymysql
import requests
from typing import Dict, List, Optional
import time

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.34
Running PyMongo Version: 4.8.0


### Extract Products Data from FakeStore API

In [7]:

def get_fakestore_products():
    """Extract products from FakeStore API"""
    try:
        response = requests.get('https://fakestoreapi.com/products')
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching FakeStore products: {e}")
        return []

#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [8]:
from pymongo import MongoClient
import json

# Example setup of logging for the notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to get MongoDB client
def get_mongo_client(host: str, port: int, username: str = None, password: str = None, db_name: str = None) -> MongoClient:
    """Initialize MongoDB client for a remote database."""
    connection_string = f"mongodb://{host}"
    if username and password:
        connection_string = f"mongodb://{username}:{password}@{host}"

    try:
        client = MongoClient(connection_string)
        if db_name:
            client = client[db_name]
        logger.info("MongoDB client initialized successfully.")
        return client
    except Exception as e:
        logger.error(f"Failed to initialize MongoDB client: {e}")
        raise e


# SQL connection
def get_sql_connection(host: str, user: str, password: str, db: str):
    """Initialize SQL connection."""
    conn = pymysql.connect(host=host, user=user, password=password, db=db)
    return conn

### Extract Data from all sources

In [12]:
# Set the path of the current working directory and append 'data' directory
data_dir = os.path.join(os.getcwd(), 'data')
logger.info(f"Data directory set to: {data_dir}")


INFO:__main__:Data directory set to: /Users/mac/Downloads/data-warehouse-project/data


In [13]:
# Define JSON files for MongoDB collections
json_files = {
    "sales_orders": 'StoreSales.json',
}


In [14]:
def set_mongo_collections(client: MongoClient, db_name: str, data_dir: str, json_files: dict):
    """Load JSON data into MongoDB collections."""
    db = client[db_name]
    for collection_name, file_name in json_files.items():
        file_path = os.path.abspath(os.path.join(data_dir, file_name))
        
        # Load JSON data and insert into MongoDB
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                if isinstance(data, list): 
                    db[collection_name].insert_many(data)
                    logger.info(f"Inserted {len(data)} documents into '{collection_name}' collection.")
                else:
                    db[collection_name].insert_one(data)
                    logger.info(f"Inserted a single document into '{collection_name}' collection.")
        except Exception as e:
            logger.error(f"Error loading data for collection '{collection_name}': {str(e)}")


In [18]:
# MongoDB connection arguments (example)
mongodb_args = {
    "host": "localhost",
    "port": 27017,
    "username": "mikelangelo1",
    "password": "password123",
    "db_name": "data_ware_house"
}

# Initialize the MongoDB client
client = get_mongo_client(
    host=mongodb_args["host"],
    port=mongodb_args["port"],
    username=mongodb_args.get("username"),
    password=mongodb_args.get("password")
)

INFO:__main__:MongoDB client initialized successfully.


#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [19]:
# Load data into MongoDB collections
set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)

INFO:__main__:Inserted 51291 documents into 'sales_orders' collection.


#### Data Extractor
This class provides mock methods to:

Extract data from a MongoDB collection.

In [ ]:

def extract_from_mongodb(collection: str, query: Dict = None) -> pd.DataFrame:
    """Extract data from MongoDB collection."""
    try:
        mongo_db = get_mongo_client(mongodb_args["host"], mongodb_args["port"], mongodb_args["username"], mongodb_args["password"], mongodb_args["db_name"])
        print("mongo_db", mongo_db)
        data = mongo_db[collection].find(query or {})  
        return pd.DataFrame(list(data))
    except Exception as e:
        logger.error(f"Error extracting from MongoDB collection '{collection}': {str(e)}")
        raise     

INFO:__main__:MongoDB client initialized successfully.


mongo_db Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'data_ware_house')
MongoDB Data:
                             _id Row ID         Order ID  Order Date  \
0       673167cbbdc428b0f7839e83  32298   CA-2012-124891  31-07-2012   
1       673167cbbdc428b0f7839e84  26341    IN-2013-77878  05-02-2013   
2       673167cbbdc428b0f7839e85  25330    IN-2013-71249  17-10-2013   
3       673167cbbdc428b0f7839e86  13524  ES-2013-1579342  28-01-2013   
4       673167cbbdc428b0f7839e87  47221     SG-2013-4320  05-11-2013   
...                          ...    ...              ...         ...   
307741  6751df929f0da4b4773d748a  35398   US-2014-102288  20-06-2014   
307742  6751df929f0da4b4773d748b  40470   US-2013-155768  02-12-2013   
307743  6751df929f0da4b4773d748c   9596   MX-2012-140767  18-02-2012   
307744  6751df929f0da4b4773d748d   6147   MX-2012-134460  22-05-2012   
307745  6751df929f0da4b4773d748e                     NaN         Na

INFO:__main__:Extracted 20 products from FakeStore API


Api Data:     id                                              title   price  \
0    1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1    2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2    3                                 Mens Cotton Jacket   55.99   
3    4                               Mens Casual Slim Fit   15.99   
4    5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   
5    6                       Solid Gold Petite Micropave   168.00   
6    7                         White Gold Plated Princess    9.99   
7    8  Pierced Owl Rose Gold Plated Stainless Steel D...   10.99   
8    9  WD 2TB Elements Portable External Hard Drive -...   64.00   
9   10  SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...  109.00   
10  11  Silicon Power 256GB SSD 3D NAND A55 SLC Cache ...  109.00   
11  12  WD 4TB Gaming Drive Works with Playstation 4 P...  114.00   
12  13  Acer SB220Q bi 21.5 inches Full HD (1920 x 108...  599.00   
13  14  Samsung 49-Inch 

### Transform Data

In [37]:
def create_unified_product_dimension(sales_df: pd.DataFrame, api_df: pd.DataFrame) -> pd.DataFrame:
    """Create unified product dimension from both sources"""
    # Transform sales products
    sales_products = sales_df[[
        'Product ID', 'Product Name', 'Category', 'Sub-Category'
    ]].drop_duplicates()
    sales_products['source'] = 'sales'
    sales_products.columns = ['product_id', 'product_name', 'category', 'subcategory', 'source']

    # Transform API products
    api_products = pd.DataFrame({
        'product_id': 'API_' + api_df['id'].astype(str),
        'product_name': api_df['title'],
        'category': api_df['category'],
        'subcategory': 'N/A',  # FakeStore API doesn't have subcategories
        'source': 'api'
    })

    # Combine products from both sources
    unified_products = pd.concat([sales_products, api_products], ignore_index=True)
    unified_products['product_key'] = range(1, len(unified_products) + 1)
    
    return unified_products

def create_fact_product_inventory(api_df: pd.DataFrame, dim_product: pd.DataFrame) -> pd.DataFrame:
    """Create product inventory fact table from API data"""
    # Extract rating data into separate columns
    api_df['rating_rate'] = api_df['rating'].apply(lambda x: x['rate'])
    api_df['rating_count'] = api_df['rating'].apply(lambda x: x['count'])

    # Create inventory fact records
    inventory_facts = pd.DataFrame({
        'product_key': dim_product[dim_product['source'] == 'api']['product_key'],
        'price': api_df['price'],
        'rating': api_df['rating_rate'],
        'rating_count': api_df['rating_count'],
        'last_updated': pd.Timestamp.now()
    })

    return inventory_facts

def create_date_dimension( df: pd.DataFrame) -> pd.DataFrame:
    """Create date dimension with proper null handling"""
    # Convert dates to datetime, handling NaN values
    order_dates = pd.to_datetime(df['Order Date'], format='%d-%m-%Y', errors='coerce')
    ship_dates = pd.to_datetime(df['Ship Date'], format='%d-%m-%Y', errors='coerce')
    
    # Combine and get unique dates, dropping NaN
    all_dates = pd.concat([order_dates, ship_dates]).dropna().unique()
    
    if len(all_dates) == 0:
        # If no valid dates, create a default date
        all_dates = [pd.Timestamp('1900-01-01')]
        
    all_dates = pd.DatetimeIndex(sorted(all_dates))
    
    # Create date dimension
    dim_date = pd.DataFrame({
        'date_id': all_dates.strftime('%Y%m%d').astype(int),
        'date': all_dates,
        'year': all_dates.year,
        'quarter': all_dates.quarter,
        'month': all_dates.month,
        'month_name': all_dates.strftime('%B'),
        'week': all_dates.isocalendar().week,
        'day': all_dates.day,
        'weekday': all_dates.dayofweek,
        'weekday_name': all_dates.strftime('%A'),
        'is_weekend': all_dates.dayofweek.isin([5, 6]).astype(int)
    })
    
    return dim_date
    
def create_customer_dimension(df: pd.DataFrame) -> pd.DataFrame:
    """Create customer dimension with proper null handling"""
    customer_cols = [
        'Customer ID', 'Customer Name', 'Segment',
        'City', 'State', 'Country', 'Market', 'Region'
    ]
    
    dim_customer = df[customer_cols].drop_duplicates()
    dim_customer = dim_customer.fillna({
        'Customer ID': 'UNKNOWN',
        'Customer Name': 'Unknown',
        'Segment': 'Unknown',
        'City': 'Unknown',
        'State': 'Unknown',
        'Country': 'Unknown',
        'Market': 'Unknown',
        'Region': 'Unknown'
    })
    
    # Rename columns
    dim_customer.columns = [
        'customer_id', 'customer_name', 'segment',
        'city', 'state', 'country', 'market', 'region'
    ]
    
    # Add customer key
    dim_customer['customer_key'] = range(1, len(dim_customer) + 1)
    
    return dim_customer
    
def create_geography_dimension(df: pd.DataFrame) -> pd.DataFrame:
    """Create geography dimension with proper null handling"""
    geo_cols = ['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']
    
    dim_geography = df[geo_cols].drop_duplicates()
    dim_geography = dim_geography.fillna({
        'City': 'Unknown',
        'State': 'Unknown',
        'Country': 'Unknown',
        'Market': 'Unknown',
        'Region': 'Unknown',
        'Postal Code': 'Unknown'
    })
    
    # Rename columns
    dim_geography.columns = [
        'city', 'state', 'country', 'market', 'region', 'postal_code'
    ]
    
    # Add geography key
    dim_geography['geo_key'] = range(1, len(dim_geography) + 1)
    
    return dim_geography
    
def create_fact_sales(df: pd.DataFrame) -> pd.DataFrame:
    """Create sales fact table from sales data using merge operations"""
    # Create a working copy
    working_df = df.copy()
    
    # Convert dates to date_id format
    working_df['order_date_id'] = pd.to_datetime(
        working_df['Order Date'], 
        format='%d-%m-%Y', 
        errors='coerce'
    ).fillna(pd.Timestamp('1900-01-01')).dt.strftime('%Y%m%d').astype(int)
    
    working_df['ship_date_id'] = pd.to_datetime(
        working_df['Ship Date'], 
        format='%d-%m-%Y', 
        errors='coerce'
    ).fillna(pd.Timestamp('1900-01-01')).dt.strftime('%Y%m%d').astype(int)
    
    # Get dimension keys through merges
    dim_customer = create_customer_dimension(working_df)
    working_df = working_df.merge(
        dim_customer[['customer_id', 'customer_key']],
        left_on='Customer ID',
        right_on='customer_id',
        how='left'
    )
    
    dim_geography = create_geography_dimension(working_df)
    working_df = working_df.merge(
        dim_geography[['city', 'state', 'country', 'geo_key']],
        left_on=['City', 'State', 'Country'],
        right_on=['city', 'state', 'country'],
        how='left'
    )
    
    # Create fact table
    fact_sales = pd.DataFrame({
        'order_id': working_df['Order ID'].fillna('UNKNOWN'),
        'order_date_id': working_df['order_date_id'],
        'ship_date_id': working_df['ship_date_id'],
        'customer_key': working_df['customer_key'].fillna(-1).astype(int),
        'product_id': working_df['Product ID'].fillna('UNKNOWN'),  # Changed from product_key to product_id
        'geo_key': working_df['geo_key'].fillna(-1).astype(int),
        'quantity': pd.to_numeric(working_df['Quantity'], errors='coerce').fillna(0),
        'sales_amount': pd.to_numeric(working_df['Sales'], errors='coerce').fillna(0),
        'discount': pd.to_numeric(working_df['Discount'], errors='coerce').fillna(0),
        'profit': pd.to_numeric(working_df['Profit'], errors='coerce').fillna(0),
        'shipping_cost': pd.to_numeric(working_df['Shipping Cost'], errors='coerce').fillna(0),
        'order_priority': working_df['Order Priority'].fillna('UNKNOWN'),
        'ship_mode': working_df['Ship Mode'].fillna('UNKNOWN')
    })
    
    # Calculate additional metrics
    fact_sales['total_amount'] = fact_sales['quantity'] * fact_sales['sales_amount']
    fact_sales['discount_amount'] = fact_sales['total_amount'] * fact_sales['discount']
    fact_sales['net_amount'] = fact_sales['total_amount'] - fact_sales['discount_amount']
    
    return fact_sales

def transform(data: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """Transform data into dimensional model"""
    logger.info("Transforming data...")

    # Create unified product dimension combining both sources
    dim_product = create_unified_product_dimension(
        data['sales_data'], 
        data['api_products']
    )

    # Create other dimensions
    dim_date = create_date_dimension(data['sales_data'])
    dim_customer = create_customer_dimension(data['sales_data'])
    dim_geography = create_geography_dimension(data['sales_data'])

    # Create fact tables
    fact_sales = create_fact_sales(data['sales_data'])
    fact_product_inventory = create_fact_product_inventory(
        data['api_products'], 
        dim_product
    )

    return {
        'dim_date': dim_date,
        'dim_customer': dim_customer,
        'dim_product': dim_product,
        'dim_geography': dim_geography,
        'fact_sales': fact_sales,
        'fact_product_inventory': fact_product_inventory
    }
    
try:
    # 1. MongoDB extraction (sales data)
    df_mongo = extract_from_mongodb("sales_orders")
    print("MongoDB Data:")
    print(df_mongo)
    
    # 2. FakeStore API extraction
    api_products = get_fakestore_products()
    api_products_df = pd.DataFrame(api_products)
    print("Api Data:", api_products_df)
    print(df_mongo)
    logger.info(f"Extracted {len(api_products_df)} products from FakeStore API")
    
    # 3. Transform data into dimensional model
    raw_data = {
        'sales_data': df_mongo,
        'api_products': api_products_df
    }
    transformed_tables = transform(raw_data)  
    [print(table) for table in transformed_tables.values()]
    
    # # 4. Load transformed data into MySQL database
    # load_to_mysql(transformed_tables)
except Exception as e:
    logger.error(f"Error in MongoDB extraction test: {str(e)}")
    


INFO:__main__:MongoDB client initialized successfully.


mongo_db Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'data_ware_house')
MongoDB Data:
                             _id Row ID         Order ID  Order Date  \
0       673167cbbdc428b0f7839e83  32298   CA-2012-124891  31-07-2012   
1       673167cbbdc428b0f7839e84  26341    IN-2013-77878  05-02-2013   
2       673167cbbdc428b0f7839e85  25330    IN-2013-71249  17-10-2013   
3       673167cbbdc428b0f7839e86  13524  ES-2013-1579342  28-01-2013   
4       673167cbbdc428b0f7839e87  47221     SG-2013-4320  05-11-2013   
...                          ...    ...              ...         ...   
307741  6751df929f0da4b4773d748a  35398   US-2014-102288  20-06-2014   
307742  6751df929f0da4b4773d748b  40470   US-2013-155768  02-12-2013   
307743  6751df929f0da4b4773d748c   9596   MX-2012-140767  18-02-2012   
307744  6751df929f0da4b4773d748d   6147   MX-2012-134460  22-05-2012   
307745  6751df929f0da4b4773d748e                     NaN         Na

INFO:__main__:Extracted 20 products from FakeStore API


Api Data:     id                                              title   price  \
0    1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1    2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2    3                                 Mens Cotton Jacket   55.99   
3    4                               Mens Casual Slim Fit   15.99   
4    5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   
5    6                       Solid Gold Petite Micropave   168.00   
6    7                         White Gold Plated Princess    9.99   
7    8  Pierced Owl Rose Gold Plated Stainless Steel D...   10.99   
8    9  WD 2TB Elements Portable External Hard Drive -...   64.00   
9   10  SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...  109.00   
10  11  Silicon Power 256GB SSD 3D NAND A55 SLC Cache ...  109.00   
11  12  WD 4TB Gaming Drive Works with Playstation 4 P...  114.00   
12  13  Acer SB220Q bi 21.5 inches Full HD (1920 x 108...  599.00   
13  14  Samsung 49-Inch 

INFO:__main__:Transforming data...


             date_id       date  year  quarter  month month_name  week  day  \
2011-01-01  20110101 2011-01-01  2011        1      1    January    52    1   
2011-01-02  20110102 2011-01-02  2011        1      1    January    52    2   
2011-01-03  20110103 2011-01-03  2011        1      1    January     1    3   
2011-01-04  20110104 2011-01-04  2011        1      1    January     1    4   
2011-01-05  20110105 2011-01-05  2011        1      1    January     1    5   
...              ...        ...   ...      ...    ...        ...   ...  ...   
2015-01-03  20150103 2015-01-03  2015        1      1    January     1    3   
2015-01-04  20150104 2015-01-04  2015        1      1    January     1    4   
2015-01-05  20150105 2015-01-05  2015        1      1    January     2    5   
2015-01-06  20150106 2015-01-06  2015        1      1    January     2    6   
2015-01-07  20150107 2015-01-07  2015        1      1    January     2    7   

            weekday weekday_name  is_weekend  
2011

## Data Loader
This class provides the implementation of load_to_warehouse, simulating loading a DataFrame into a data warehouse. It connects to the MSQL database and load data


In [ ]:
import logging
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine


class MsqlDatabaseConnection:
    def __init__(self, username: str, password: str, host: str = "localhost", port: int = 3306, db_name: str = "myshop"):
        self.username = username
        self.password = password
        self.host = host
        self.port = port
        self.db_name = db_name

    def get_sqlalchemy_engine(self) -> Engine:
        """Return a SQLAlchemy engine connected to MySQL database with UTF-8 support."""
        try:
            connection_uri = f"mysql+pymysql://{self.username}:{self.password}@{self.host}:{self.port}/{self.db_name}?charset=utf8mb4"
            engine = create_engine(
                connection_uri,
                connect_args={'charset': 'utf8mb4'}
            )
            logger.info("Successfully connected to MySQL database.")
            return engine
        except Exception as e:
            logger.error(f"Error connecting to MySQL: {str(e)}")
            raise
    
    def get_query_result(self, query: str) -> pd.DataFrame:
        """Execute a query and return the result as a DataFrame."""
        try:
            engine = self.get_sqlalchemy_engine()
            result = pd.read_sql(query, engine)
            logger.info(f"Query executed successfully: {query}")
            return result
        except Exception as e:
            logger.error(f"Error executing query: {str(e)}")
            raise

# DataLoader class
class DataLoader:
    def __init__(self, db_connection: MsqlDatabaseConnection):
        self.db_conn = db_connection
    
    def load_to_warehouse(self, df: pd.DataFrame, table_name: str, if_exists: str = 'append') -> None:
        """Load DataFrame to data warehouse."""
        try:
            engine = self.db_conn.get_sqlalchemy_engine()
            df.to_sql(
                name=table_name,
                con=engine,
                if_exists=if_exists,
                index=False,
                chunksize=1000
            )
            logger.info(f"Successfully loaded {len(df)} rows to {table_name}")
        except Exception as e:
            logger.error(f"Error loading data to warehouse: {str(e)}")
            raise


db_connection = MsqlDatabaseConnection(
    username="root",  # Replace with your MySQL username
    password="Akinolami6650!",  # Replace with your MySQL password
    db_name="myshop"           # The name of your database
)

# Create an instance of DataLoader
data_loader = DataLoader(db_connection)

In [41]:
def load_to_mysql(tables: Dict[str, pd.DataFrame], data_loader: DataLoader):
    """Load all tables into MySQL warehouse"""
    try:
        # Define loading order (dimensions first, then facts)
        load_order = [
            'dim_date',
            'dim_customer',
            'dim_product',
            'dim_geography',
            'fact_sales',
            'fact_product_inventory'
        ]

        for table_name in load_order:
            if table_name in tables:
                logger.info(f"Loading {table_name}...")
                data_loader.load_to_warehouse(
                    df=tables[table_name],
                    table_name=table_name,
                    if_exists='replace'  # Using replace to refresh the tables
                )
                
    except Exception as e:
        logger.error(f"Error in load_to_mysql: {e}")
        raise

try:
    # 1. MongoDB extraction (sales data)
    df_mongo = extract_from_mongodb("sales_orders")
    print("MongoDB Data extracted successfully")
    
    # 2. FakeStore API extraction
    api_products = get_fakestore_products()
    api_products_df = pd.DataFrame(api_products)
    print("API Data extracted successfully")
    
    # 3. Transform data into dimensional model
    raw_data = {
        'sales_data': df_mongo,
        'api_products': api_products_df
    }
    transformed_tables = transform(raw_data)
    print("Data transformation completed")
    
    # 4. Load transformed data into MySQL database
    db_connection = MsqlDatabaseConnection(
        username="root",
        password="Akinolami6650!",
        db_name="myshop"
    )
    data_loader = DataLoader(db_connection)
    
    # Load the data
    load_to_mysql(transformed_tables, data_loader)
    print("Data loading completed")
    
    # 5. Verify the load with some basic queries
    verification_queries = {
        "Customer Count": "SELECT COUNT(*) as customer_count FROM dim_customer",
        "Product Count": "SELECT COUNT(*) as product_count FROM dim_product",
        "Total Sales": "SELECT COUNT(*) as sales_count, SUM(sales_amount) as total_sales FROM fact_sales"
    }
    
    print("\nVerification Results:")
    for name, query in verification_queries.items():
        result = db_connection.get_query_result(query)
        print(f"\n{name}:")
        print(result)

except Exception as e:
    logger.error(f"Error in ETL pipeline: {str(e)}")

INFO:__main__:MongoDB client initialized successfully.


mongo_db Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'data_ware_house')
MongoDB Data extracted successfully


INFO:__main__:Transforming data...


API Data extracted successfully


INFO:__main__:Loading dim_date...
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Successfully loaded 1468 rows to dim_date
INFO:__main__:Loading dim_customer...
INFO:__main__:Successfully connected to MySQL database.


Data transformation completed


INFO:__main__:Successfully loaded 24960 rows to dim_customer
INFO:__main__:Loading dim_product...
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Successfully loaded 10789 rows to dim_product
INFO:__main__:Loading dim_geography...
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Successfully loaded 3848 rows to dim_geography
INFO:__main__:Loading fact_sales...
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Successfully loaded 8896128 rows to fact_sales
INFO:__main__:Loading fact_product_inventory...
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Successfully loaded 40 rows to fact_product_inventory
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Query executed successfully: SELECT COUNT(*) as customer_count FROM dim_customer
INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Query executed successfully: SELECT COUNT(*) as product_count FROM dim_product
INFO:

Data loading completed

Verification Results:

Customer Count:
   customer_count
0           24960

Product Count:
   product_count
0          10789


INFO:__main__:Query executed successfully: SELECT COUNT(*) as sales_count, SUM(sales_amount) as total_sales FROM fact_sales



Total Sales:
   sales_count   total_sales
0      8896128  2.262738e+09


## Sales Analysis by Customer Segment and Quarter

In [42]:
#1. Sales Performance Analysis by Time Period
sales_by_segment_query = """
SELECT 
    d.year,
    d.quarter,
    COUNT(DISTINCT fs.order_id) as total_orders,
    COUNT(DISTINCT fs.customer_key) as unique_customers,
    SUM(fs.sales_amount) as total_sales,
    AVG(fs.sales_amount) as avg_sale_per_order,
    STDDEV(fs.sales_amount) as sales_std_dev,
    MIN(fs.sales_amount) as min_sale,
    MAX(fs.sales_amount) as max_sale,
    SUM(fs.profit) as total_profit,
    AVG(fs.profit) as avg_profit_per_order,
    STDDEV(fs.profit) as profit_std_dev,
    AVG(fs.discount) * 100 as avg_discount_percentage,
    SUM(fs.shipping_cost) as total_shipping_cost
FROM fact_sales fs
JOIN dim_date d ON fs.order_date_id = d.date_id
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter;
"""

try:
    df_segment_analysis = db_connection.get_query_result(sales_by_segment_query)
    print("Sales Analysis by Customer Segment and Quarter:")
    print(df_segment_analysis)
except Exception as e:
    logger.error(f"Error in segment analysis query: {str(e)}")


INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Query executed successfully: 
SELECT 
    d.year,
    d.quarter,
    COUNT(DISTINCT fs.order_id) as total_orders,
    COUNT(DISTINCT fs.customer_key) as unique_customers,
    SUM(fs.sales_amount) as total_sales,
    AVG(fs.sales_amount) as avg_sale_per_order,
    STDDEV(fs.sales_amount) as sales_std_dev,
    MIN(fs.sales_amount) as min_sale,
    MAX(fs.sales_amount) as max_sale,
    SUM(fs.profit) as total_profit,
    AVG(fs.profit) as avg_profit_per_order,
    STDDEV(fs.profit) as profit_std_dev,
    AVG(fs.discount) * 100 as avg_discount_percentage,
    SUM(fs.shipping_cost) as total_shipping_cost
FROM fact_sales fs
JOIN dim_date d ON fs.order_date_id = d.date_id
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter;



Sales Analysis by Customer Segment and Quarter:
    year  quarter  total_orders  unique_customers   total_sales  \
0   2011        1           673             10745  5.264300e+07   
1   2011        2          1027             14062  8.451945e+07   
2   2011        3          1217             15664  1.164795e+08   
3   2011        4          1581             18100  1.580572e+08   
4   2012        1           826             12663  7.226945e+07   
5   2012        2          1297             16453  1.067034e+08   
6   2012        3          1480             17552  1.338895e+08   
7   2012        4          1837             18945  1.722617e+08   
8   2013        1          1059             14485  9.578419e+07   
9   2013        2          1635             18361  1.524045e+08   
10  2013        3          1962             19848  1.647233e+08   
11  2013        4          2184             19978  1.994881e+08   
12  2014        1          1355             16845  1.195761e+08   
13  2014      

##  Customer Segment Performance Analysis

In [56]:

product_performance_query = """
SELECT 
    dp.category,
    dp.subcategory,
    COUNT(DISTINCT fs.order_id) as total_orders,
    SUM(fs.quantity) as total_units_sold,
    AVG(fs.quantity) as avg_units_per_order,
    STDDEV(fs.quantity) as quantity_std_dev,
    SUM(fs.sales_amount) as total_revenue,
    AVG(fs.sales_amount) as avg_revenue_per_order,
    STDDEV(fs.sales_amount) as revenue_std_dev,
    AVG(fpi.rating) as avg_product_rating,
    STDDEV(fpi.rating) as rating_std_dev,
    AVG(fpi.price) as avg_list_price,
    STDDEV(fpi.price) as price_std_dev
FROM dim_product dp
LEFT JOIN fact_sales fs ON dp.product_id = fs.product_id  -- Changed from product_key to product_id
LEFT JOIN fact_product_inventory fpi ON dp.product_key = fpi.product_key
GROUP BY dp.category, dp.subcategory
ORDER BY total_revenue DESC;
"""

try:
    df_segment_analysis = db_connection.get_query_result(product_performance_query)
    print("Product Performance and Inventory Analysis:")
    print(df_segment_analysis)
except Exception as e:
    logger.error(f"Error in product performance analysis query: {str(e)}")



INFO:__main__:Successfully connected to MySQL database.
ERROR:__main__:Error executing query: (pymysql.err.InternalError) (3, "Error writing file '/var/tmp/MYbUq5Pn' (OS errno 28 - No space left on device)")
[SQL: 
SELECT 
    dp.category,
    dp.subcategory,
    COUNT(DISTINCT fs.order_id) as total_orders,
    SUM(fs.quantity) as total_units_sold,
    AVG(fs.quantity) as avg_units_per_order,
    STDDEV(fs.quantity) as quantity_std_dev,
    SUM(fs.sales_amount) as total_revenue,
    AVG(fs.sales_amount) as avg_revenue_per_order,
    STDDEV(fs.sales_amount) as revenue_std_dev,
    AVG(fpi.rating) as avg_product_rating,
    STDDEV(fpi.rating) as rating_std_dev,
    AVG(fpi.price) as avg_list_price,
    STDDEV(fpi.price) as price_std_dev
FROM dim_product dp
LEFT JOIN fact_sales fs ON dp.product_id = fs.product_id  -- Changed from product_key to product_id
LEFT JOIN fact_product_inventory fpi ON dp.product_key = fpi.product_key
GROUP BY dp.category, dp.subcategory
ORDER BY total_revenue DE

## Geographic Sales Distribution

In [57]:

geographic_sales_query = """
SELECT 
    c.country_region as Country,
    c.city as City,
    COUNT(DISTINCT o.customer_id) as Unique_Customers,
    SUM(od.quantity * od.unit_price * (1 - od.discount)) as Total_Sales,
    AVG(od.quantity * od.unit_price * (1 - od.discount)) as Avg_Order_Value,
    SUM(od.quantity) as Total_Units_Sold
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN order_details od ON o.id = od.order_id
GROUP BY c.country_region, c.city
HAVING Total_Sales > 1000
ORDER BY Total_Sales DESC;
"""

try:
    df_geographic_sales = db_connection.get_query_result(geographic_sales_query)
    print("\nGeographic Sales Distribution:")
    print(df_geographic_sales)
except Exception as e:
    logger.error(f"Error in geographic sales query: {str(e)}")

#

INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Query executed successfully: 
SELECT 
    c.country_region as Country,
    c.city as City,
    COUNT(DISTINCT o.customer_id) as Unique_Customers,
    SUM(od.quantity * od.unit_price * (1 - od.discount)) as Total_Sales,
    AVG(od.quantity * od.unit_price * (1 - od.discount)) as Avg_Order_Value,
    SUM(od.quantity) as Total_Units_Sold
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN order_details od ON o.id = od.order_id
GROUP BY c.country_region, c.city
HAVING Total_Sales > 1000
ORDER BY Total_Sales DESC;




Geographic Sales Distribution:
   Country            City  Unique_Customers  Total_Sales  Avg_Order_Value  \
0      USA         Memphis                 1     15432.50      3858.125000   
1      USA           Boise                 1     13800.00     13800.000000   
2      USA       Milwaukee                 1      8007.50      1334.583333   
3      USA        New York                 1      4949.00       707.000000   
4      USA        Portland                 1      4683.00       780.500000   
5      USA           Miami                 2      4644.75       663.535714   
6      USA  Salt Lake City                 1      3786.50      1262.166667   
7      USA          Denver                 1      2905.50       968.500000   
8      USA       Las Vegas                 2      2695.00       673.750000   
9      USA     Los Angelas                 1      2550.00       510.000000   
10     USA         Seattle                 1      2410.75       602.687500   
11     USA         Chicago      

## Shipping Performance

In [59]:
shipping_performance_query = """
SELECT 
    s.company as Ship_Mode,
    COUNT(DISTINCT o.id) as Total_Shipments,
    AVG(DATEDIFF(o.shipped_date, o.order_date)) as Avg_Days_to_Ship,
    SUM(o.shipping_fee) as Total_Shipping_Cost,
    SUM(od.quantity * od.unit_price * (1 - od.discount)) as Total_Order_Value,
    (SUM(o.shipping_fee) / SUM(od.quantity * od.unit_price * (1 - od.discount))) * 100 as Shipping_Cost_Percentage
FROM orders o
JOIN shippers s ON o.shipper_id = s.id
JOIN order_details od ON o.id = od.order_id
WHERE o.shipped_date IS NOT NULL
GROUP BY s.company
ORDER BY Total_Shipments DESC;
"""

try:
    df_shipping_performance = db_connection.get_query_result(shipping_performance_query)
    print("\nShipping Performance Analysis:")
    print(df_shipping_performance)
except Exception as e:
    logger.error(f"Error in shipping performance query: {str(e)}")



INFO:__main__:Successfully connected to MySQL database.
INFO:__main__:Query executed successfully: 
SELECT 
    s.company as Ship_Mode,
    COUNT(DISTINCT o.id) as Total_Shipments,
    AVG(DATEDIFF(o.shipped_date, o.order_date)) as Avg_Days_to_Ship,
    SUM(o.shipping_fee) as Total_Shipping_Cost,
    SUM(od.quantity * od.unit_price * (1 - od.discount)) as Total_Order_Value,
    (SUM(o.shipping_fee) / SUM(od.quantity * od.unit_price * (1 - od.discount))) * 100 as Shipping_Cost_Percentage
FROM orders o
JOIN shippers s ON o.shipper_id = s.id
JOIN order_details od ON o.id = od.order_id
WHERE o.shipped_date IS NOT NULL
GROUP BY s.company
ORDER BY Total_Shipments DESC;




Shipping Performance Analysis:
            Ship_Mode  Total_Shipments  Avg_Days_to_Ship  Total_Shipping_Cost  \
0  Shipping Company B               14            1.1667               1618.0   
1  Shipping Company C               11            0.3333                479.0   
2  Shipping Company A                7            3.6923                335.0   

   Total_Order_Value  Shipping_Cost_Percentage  
0           16078.50                 10.063128  
1           24802.25                  1.931276  
2            9593.50                  3.491948  
